# Random Forest 4.2

Implementación más profesional del random forest con gridsearch y cross validation, con los datos de tiempos. <br>
**nota:** ya no se fijará la random seed. Se usará un split predefinido.

In [ ]:
# paquetes
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import make_scorer, recall_score, confusion_matrix, classification_report
import time

In [ ]:
# importamos los datos
df_import = pd.read_csv('df_datos_tiempos.csv', index_col='Unnamed: 0')
train_ids = list(pd.read_csv('splits/train_ids634.csv', header=None)[0])
test_ids = list(pd.read_csv('splits/test_ids634.csv', header=None)[0])

# llenamos los train-test splits
df_train = df_import[df_import['id_ecg'].isin(train_ids)].copy()
df_test = df_import[df_import['id_ecg'].isin(test_ids)].copy()

# separamos las variables predictivas
X_train = df_train.drop(columns = ['id_ecg','categoria','patient_id']).copy()
X_test = df_test.drop(columns = ['id_ecg','categoria','patient_id']).copy()

# codificamos las categorías
label_encoder = LabelEncoder()
categorias = df_import['categoria'].unique()
label_encoder.fit(categorias)

# generamos la variable objetivo
y_train = label_encoder.transform(df_train['categoria'])
y_test = label_encoder.transform(df_test['categoria'])

# vamos a imprimir los resultados del splitting
train_size = X_train.shape[0]
test_size = X_test.shape[0]
porcentaje_train = train_size/(train_size+test_size)
porcentaje_test = test_size/(train_size+test_size)
print()
print(f'\033[1m tenemos {train_size:d} eventos para el training ({porcentaje_train:.2%}). \033[0m') # f-strings y negritas
print()
print(f'\033[1m tenemos {test_size:d} eventos para el testing ({porcentaje_test:.2%}). \033[0m') # f-strings y negritas
print()

In [ ]:
# creamos el modelo abstracto de gridsearch con cross validation

# definimos el random forest. Los parámetros se definirán en el gridsearch
rf_abstracto = RandomForestClassifier()

# método de foldeo
cross_validation_strategy = StratifiedKFold(n_splits=5, # reducimos el número de splits para optimizar el proceso
                                            shuffle=True)

# método de scoring por recall
recall_scorer = make_scorer(recall_score, average='macro')

# hacemos cálculos para hacer una lista artesanal de posibles valores en max_features.
n_features = X_train.shape[1] # vemos cuantas features tenemos
sqrt_n = np.round(np.sqrt(n_features),0) # calculamos la raíz cuadrada y redondeamos
raiz_n = np.round(np.log2(n_features),0) # calculamos el logaritmo base 2 y redondeamos
otros_valores = [1,5,10,15,20,25] # mas valores manuales para max_features
opciones_max_features = sorted(list(set([n_features,sqrt_n,raiz_n]+otros_valores))) # generamos una lista ordenada y sin repetidos
opciones_max_features = [int(x) for x in opciones_max_features] # para evitar errores de formato, pasamos a int

# hacemos el diccionario de hiperparámetros para el gridsearch
parametros_gridsearch = {'n_estimators':[25,50,75,100,125,150], # hiperparámetro a modificar 1: número de árboles
                         'max_features':opciones_max_features, # hiperparámetro a modificar 2: número de variables para la cosntrucción de árboles
                         'min_samples_split':[2,20,40,60]} # hiperparámetro a modificar 3: número de eventos para dividir rama

#gridsearch: ejecutar CrossValidation con diferentes parámetros.
gridsearch_abstracto = GridSearchCV(estimator=rf_abstracto,
                                    param_grid=parametros_gridsearch, # usamos el diccionario creado arriba
                                    scoring=recall_scorer, # usamos el método de scoring por recall
                                    n_jobs=-2, # -2 indica que todos los procesadores serán usados menos uno.
                                    refit=False, # NO recalculará un modelo con los mejores parámetros y todos los datos.
                                    cv=cross_validation_strategy,
                                    error_score='raise')

In [ ]:
# iniciamos el timer
tiempo_inicio = time.time()

# ejecutamos gridsearch con las bases de datos
gridsearch = gridsearch_abstracto.fit(X_train, y_train)

# concluimos el timer
tiempo_final = time.time()

print(f'la ejecución del gridsearch tomó {(tiempo_final-tiempo_inicio)/60:.2f} minutos.')

In [ ]:
# vamos a visualizar los resultados

# guardamos los resultados que nos interesan
gridsearch_resultados = gridsearch.cv_results_ # aquí están todos los resultados resumidos
parametros = gridsearch_resultados['params'] # guardamos los parámetros y...
resultados = gridsearch_resultados['mean_test_score'] # el resultado del modelo con esos parámetros.

# guardamos el i-esimo resultado con sus parámetros
for i, dic in enumerate(parametros):
    dic.update({'accuracy':resultados[i]})

# vemos cuales parametros tenemos
valores_n_estimators = np.unique([p['n_estimators'] for p in parametros])
valores_max_features = np.unique([p['max_features'] for p in parametros])
valores_min_samples_split = np.unique([p['min_samples_split'] for p in parametros])

# vamos a hacer muchas graficas con los diferentes valores de min_samples_split con un loop
matriz0 = np.zeros((len(valores_n_estimators),len(valores_max_features)))

for k, split in enumerate(valores_min_samples_split):
    
    #creamos el dataframe con ceros
    df_resultados = pd.DataFrame(data = matriz0,
                                 index=valores_n_estimators,
                                 columns=valores_max_features)
    
    # llenamos el dataframe
    for i, n_estimator in enumerate(valores_n_estimators):
        for j, max_feature in enumerate(valores_max_features):
            df_resultados.iloc[i,j] = [x['accuracy'] for x in parametros 
                                       if (x['min_samples_split']==split) 
                                       and (x['n_estimators']==n_estimator) 
                                       and (x['max_features']==max_feature)][0]
    
    # graficamos
    plt.figure(figsize=(5, 5))
    sns.heatmap(df_resultados.iloc[::-1], cmap='Paired', vmin=0.45, vmax=0.55)
    plt.title('min samples split = '+str(split))
    plt.ylabel("Número de árboles")
    plt.xlabel("Número de variables")
    plt.show()
    plt.close()

In [ ]:
# rearmamos el mejor modelo
mejor_modelo_abstracto = RandomForestClassifier(n_estimators=gridsearch.best_params_['n_estimators'],
                                                criterion='gini',
                                                max_depth=None,
                                                min_samples_split=gridsearch.best_params_['min_samples_split'],
                                                max_features=gridsearch.best_params_['max_features'],
                                                bootstrap=True)
# entrenamos el mejor modelo
mejor_modelo = mejor_modelo_abstracto.fit(X=X_train, y=y_train)

# predecimos con el mejor modelo
y_prediccion = mejor_modelo.predict(X_test)

# calculamos los resultados
matriz_confusion = confusion_matrix(y_true=y_test, y_pred=y_prediccion)

# imprimimos los parametros optimos
df_mejor_modelo = pd.DataFrame(list(gridsearch.best_params_.items()),
                               columns=['parametro','valor óptimo'])
print(df_mejor_modelo.to_string(index=False))

In [ ]:
# imprimimos la matriz de confusion
plt.figure(figsize=(8, 8))
sns.heatmap(matriz_confusion,
            annot=True, # anotar valor en cada cuadro
            fmt='d', # formato del valor que se mostrará en cada cuadro. 'd' hace referencia a 'sin decimales'
            cmap='Blues', # le estamos diciendo que use colores azules
            xticklabels=label_encoder.classes_, # leyenda del eje x
            yticklabels=label_encoder.classes_) # leyenda del eje y
plt.xlabel('Predicción') # título eje x
plt.ylabel('Condición Real') # título eje y
plt.title('Matriz de Confusión - Diagnóstico ECG') # título del gráfico
plt.show()

In [ ]:
# tabla de resultados
# precision = True Positive / (True Positive + False Positive) = De los que predijiste con padecimiento X, cuántos si tienen el padecimiento.
# recall = True Positive / (True Positive + False Negative) = De los que tienen padecimiento X, cuántos predijiste correctamente.
# f1-score = Media armónica de precisión y recall = Penaliza valores bajos, por lo que ambos valores deben ser buenos para un valor alto.
# support = tamaño de muestra
# acurracy = precisión global = (Global True Positives + Global True Negatives) / number of events
# macro average = promedio de la columna
# weighted average = promedio de la columna ponderado por su tamaño de muestra
print("                 " + "\033[1m" + 'Reporte de clasificación' + "\033[1m")
print(classification_report(y_true=y_test, y_pred=y_prediccion, target_names=label_encoder.classes_))
print("(accuracy en cross-validation: " + str(gridsearch.best_score_) + ")")